In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats as stat
from matplotlib import pyplot as plt

In [2]:
veh_info_df=pd.read_csv('vehicles_us.csv')
print(veh_info_df)

       price  model_year           model  condition  cylinders fuel  odometer  \
0       9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500         NaN      ford f-150       good        6.0  gas   88705.0   
2       5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249      2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700      2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950      2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455      2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300      2014.0   nissan altima       good        4.0  gas       NaN   

      transmission    type 

In [3]:
veh_info_df['date_posted']=pd.to_datetime(veh_info_df['date_posted'],format='%Y-%m-%d')
veh_info_df['year']=veh_info_df['date_posted'].dt.year
veh_info_df['age']=veh_info_df['year']-veh_info_df['model_year']

In [4]:
veh_info_df.info()

veh_info_df=veh_info_df.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    47906 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  year          51525 non-null  int64         
 14  age           47906 non-null  float64       
dtypes: datetime64[ns](1), float64(5), in

In [5]:
veh_info_df=veh_info_df.astype({'price':'float64','model_year':'float64','cylinders':'float64','odometer':'float64','is_4wd':'float64','days_listed':'float64'})
veh_info_df=veh_info_df.astype({'price':'int64','model_year':'int64','cylinders':'int64','is_4wd':'int64','days_listed':'int64'})

In [6]:
veh_info_df['model'] = veh_info_df['model'].astype(str)
veh_info_df['make'] = veh_info_df['model'].apply(lambda x: x.split()[0])

In [7]:
print(veh_info_df['make'].unique())

['chrysler' 'honda' 'chevrolet' 'gmc' 'ram' 'jeep' 'subaru' 'ford' 'dodge'
 'nissan' 'toyota' 'bmw' 'hyundai' 'kia' 'buick' 'volkswagen' 'cadillac'
 'acura']


In [8]:
unique_model=veh_info_df['model'].unique()
print(sorted(unique_model))

['acura tl', 'bmw x5', 'buick enclave', 'cadillac escalade', 'chevrolet camaro', 'chevrolet colorado', 'chevrolet corvette', 'chevrolet cruze', 'chevrolet equinox', 'chevrolet malibu', 'chevrolet silverado', 'chevrolet silverado 1500', 'chevrolet silverado 1500 crew', 'chevrolet silverado 2500hd', 'chevrolet silverado 3500hd', 'chevrolet suburban', 'chevrolet tahoe', 'chevrolet trailblazer', 'chevrolet traverse', 'chrysler 200', 'chrysler 300', 'chrysler town & country', 'dodge charger', 'dodge dakota', 'dodge grand caravan', 'ford edge', 'ford escape', 'ford expedition', 'ford explorer', 'ford f-150', 'ford f-250', 'ford f-250 sd', 'ford f-250 super duty', 'ford f-350 sd', 'ford f150', 'ford f150 supercrew cab xlt', 'ford f250', 'ford f250 super duty', 'ford f350', 'ford f350 super duty', 'ford focus', 'ford fusion', 'ford fusion se', 'ford mustang', 'ford ranger', 'ford taurus', 'gmc acadia', 'gmc sierra', 'gmc sierra 1500', 'gmc sierra 2500hd', 'gmc yukon', 'honda accord', 'honda ci

In [9]:
f150_wrong_names=['ford f-150']
f250_wrong_names=['ford f-250', 'ford f-250 sd', 'ford f-250 super duty','ford f250 super duty']
f350_wrong_names=['ford f-350 sd','ford f350 super duty']
f150='ford f150'
f250='ford f250'
f350='ford f350'

In [10]:
def replace_wrong_values(wrong_values, correct_value):
    for wrong_value in wrong_values:
        veh_info_df['model'] = veh_info_df['model'].replace(wrong_value, correct_value)

replace_wrong_values(f150_wrong_names, f150)
replace_wrong_values(f250_wrong_names, f250)
replace_wrong_values(f350_wrong_names, f350)

In [11]:
excellent_car=veh_info_df[veh_info_df['condition']=='excellent']
domesetic_maker=['chrysler','chevrolet','gmc','ram', 'jeep','ford','dodge', 'buick','cadillac']
excellent_car_dom=excellent_car.query("make in @domesetic_maker")
excellent_suv_dom=excellent_car_dom[excellent_car_dom['type']=='SUV']

international_maker=['honda','subaru','nissan','toyota','bmw','hyundai','kia','volkswagen','acura']
excellent_car_int=excellent_car.query("make in @international_maker")
excellent_suv_int=excellent_car_int[excellent_car_int['type']=='SUV']

Analyzing the mean posting days between domestic and foreign SUV make and their model years

In [12]:
dom_time=excellent_suv_dom.groupby(['make','model_year'])['days_listed'].mean().reset_index()
int_time=excellent_suv_int.groupby(['make','model_year'])['days_listed'].mean().reset_index()

In [13]:
fig_dom_time = px.histogram(dom_time, x="model_year", y="days_listed",color='make', barmode='group', labels={"model_year":"Year", "days_listed":"Days"}, title="Days vs SUV Make and Year")
fig_dom_time.show()

In [14]:
fig_int_time = px.histogram(int_time, x="model_year", y="days_listed",color='make', barmode='group', labels={"model_year":"Year", "days_listed":"Days"}, title="Days vs SUV Make and Year")
fig_int_time.show()

In [15]:
dom_time_mean=dom_time['days_listed'].mean()
int_time_mean=int_time['days_listed'].mean()
dom_time_var = np.var(dom_time['days_listed'])
int_time_var = np.var(int_time['days_listed'])
dom_time_std=np.std(dom_time['days_listed'])
int_time_std=np.std(int_time['days_listed'])

N0: Foreign Make SUV listing dates are different than domestic
N1: Foeign and Domestic SUV make listing dates are similar

In [16]:
alpha = 0.05

results = stat.ttest_ind(dom_time['days_listed'], int_time['days_listed'], equal_var=False)

print('p-value:', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

p-value: 0.9815402577061809
We can't reject the null hypothesis


Analyzing the mean price between domestic and foreign SUV make and their model years

In [17]:
dom_price=excellent_suv_dom.groupby(['make','model_year'])['price'].mean().reset_index()
int_price=excellent_suv_int.groupby(['make','model_year'])['price'].mean().reset_index()

In [18]:
fig_dom_price = px.scatter(dom_price, x="model_year", y="price",color='make', labels={"model_year":"Year", "price":"USD"}, title="Price vs SUV Make and Year")
fig_dom_price.show()

In [19]:
fig_int_price = px.scatter(int_price, x="model_year", y="price",color='make', labels={"model_year":"Year", "price":"USD"}, title="Price vs SUV Make and Year")
fig_int_price.show()

In [20]:
dom_price_mean=dom_price['price'].mean()
int_price_mean=int_price['price'].mean()
dom_price_var = np.var(dom_price['price'])
int_price_var = np.var(int_price['price'])
dom_price_std=np.std(dom_price['price'])
int_price_std=np.std(int_price['price'])

N0: Foreign Make SUV prices are different than domestic
N1: Foeign and Domestic SUV make prices are similar

In [21]:
alpha = 0.05

results = stat.ttest_ind(dom_price['price'], int_price['price'], equal_var=False)

print('p-value:', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

p-value: 0.06528166427073799
We can't reject the null hypothesis
